### Launch Slurm

In [2]:
import os
import subprocess
import numpy as np
import random

In [3]:
data_dir = "/large_experiments/3po/data/arctic/arctic/data/arctic_data/data"
save_dir = "/private/home/taoshaf/data/arctic"
mocap_dir = os.path.join(data_dir, "mocap_npy")
seqs = [
    "/".join(seq.split(".npy")[0].split("_", 1))
    for seq in sorted(os.listdir(mocap_dir))
]

In [4]:
num_seqs = len(seqs)
num_jobs = min(len(seqs), 128)
num_seqs_per_job = num_seqs // num_jobs
seqs_for_jobs = [
    num_seqs_per_job + (job < (num_seqs - num_seqs_per_job * num_jobs))
    for job in range(num_jobs)
]
seq_offsets = np.cumsum([0] + seqs_for_jobs)

In [5]:
common_cmd = [
    "sbatch",
    "launch/submit_jobs.bash",
    "runs/demo/run_on_arctic.py",
    "+cam_idxs=[2]",
]

for job in range(1, num_jobs):
    seqs_job = ",".join(seqs[seq_offsets[job] : seq_offsets[job + 1]])
    cmd = common_cmd + [f"+seqs='{seqs_job}'"]
    print(f"{job}: " + " ".join(cmd))
    subprocess.run(
        cmd, cwd="/private/home/taoshaf/Documents/projects/CondDenseDetection"
    )

1: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/capsulemachine_grab_01,s01/capsulemachine_use_01,s01/capsulemachine_use_02'
Submitted batch job 52919137
2: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/espressomachine_grab_01,s01/espressomachine_use_01,s01/espressomachine_use_02'
Submitted batch job 52919138
3: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/ketchup_grab_01,s01/ketchup_use_01,s01/ketchup_use_02'
Submitted batch job 52919139
4: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/laptop_grab_01,s01/laptop_use_01,s01/laptop_use_02'
Submitted batch job 52919140
5: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/laptop_use_03,s01/laptop_use_04,s01/microwave_grab_01'
Submitted batch job 52919141
6: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/microwave_use_01,s01/m

Submitted batch job 52919145
10: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/scissors_use_02,s01/waffleiron_grab_01,s01/waffleiron_use_01'
Submitted batch job 52919146
11: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s01/waffleiron_use_02,s02/box_grab_01,s02/box_use_01'
Submitted batch job 52919147
12: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s02/box_use_02,s02/capsulemachine_grab_01,s02/capsulemachine_use_01'
Submitted batch job 52919148
13: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s02/capsulemachine_use_02,s02/espressomachine_grab_01,s02/espressomachine_use_01'
Submitted batch job 52919149
14: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s02/espressomachine_use_02,s02/ketchup_grab_01,s02/ketchup_use_01'
Submitted batch job 52919150
15: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[

sbatch: error: Slurm temporarily unable to accept job, sleeping and retrying


Submitted batch job 52919262
110: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/mixer_use_03,s09/mixer_use_04'
Submitted batch job 52919263
111: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/notebook_use_01,s09/notebook_use_02'
Submitted batch job 52919264
112: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/notebook_use_03,s09/notebook_use_04'
Submitted batch job 52919265
113: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/phone_use_02,s09/phone_use_03'


sbatch: error: Slurm temporarily unable to accept job, sleeping and retrying


Submitted batch job 52919276
114: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/scissors_use_01,s09/scissors_use_02'
Submitted batch job 52919277
115: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/scissors_use_04,s09/waffleiron_use_01'
Submitted batch job 52919278
116: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/waffleiron_use_02,s09/waffleiron_use_03'
Submitted batch job 52919279
117: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s09/waffleiron_use_04,s10/box_use_01'
Submitted batch job 52919280
118: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/box_use_02,s10/capsulemachine_use_01'
Submitted batch job 52919281
119: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/capsulemachine_use_02,s10/espressomachine_use_02'
Submitted batch job 52919282
120: sbatch launch/submit_jobs

sbatch: error: Slurm temporarily unable to accept job, sleeping and retrying


Submitted batch job 52919286
122: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/laptop_use_02,s10/mixer_use_01'
Submitted batch job 52919287
123: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/mixer_use_02,s10/notebook_use_01'
Submitted batch job 52919288
124: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/notebook_use_02,s10/phone_grab_01'
Submitted batch job 52919289
125: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/phone_use_01_retake,s10/phone_use_02'
Submitted batch job 52919290
126: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/scissors_use_02,s10/waffleiron_grab_01'
Submitted batch job 52919291
127: sbatch launch/submit_jobs.bash runs/demo/run_on_arctic.py +cam_idxs=[2] +seqs='s10/waffleiron_use_01,s10/waffleiron_use_02'
Submitted batch job 52919292


### Check Dense Keypoints

In [28]:
import os
import subprocess
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [4]:
orig_dir = "/private/home/taoshaf/data/egohumans"
anno_dir = "/private/home/taoshaf/data/annotation/egohumans/annos/final/sparse"
save_dir = "/checkpoint/taoshaf/results/CondDenseDetection/egohumans"

In [5]:
subjects = sorted(os.listdir(anno_dir))

In [49]:
seqs = []
for subject in subjects:
    anno_subject_dir = os.path.join(anno_dir, subject)
    subseqs = sorted(os.listdir(anno_subject_dir))
    seqs.extend([os.path.join(subject, subseq) for subseq in subseqs])
    
checks = []
for seq in tqdm(seqs):
    seq_dir = os.path.join(anno_dir, seq)
    arias = sorted(os.listdir(seq_dir))
    for person in arias:
        cams = sorted(os.listdir(os.path.join(seq_dir, person)))
        for cam in cams:
            try:
                num_annos = len(os.listdir(os.path.join(seq_dir, person, cam)))
                num_frames = len(os.listdir(os.path.join(save_dir, seq, person, cam)))
                if num_frames != num_annos:
                    checks.append([seq, person, cam])
            except:
                checks.append([seq, person, cam])

100%|██████████| 113/113 [00:07<00:00, 16.01it/s]


In [48]:
common_cmds = [
    "sbatch",
    "launch/submit_jobs.bash",
    "runs/demo/run_on_egohumans.py",
]

for job, (seq, person, cam) in enumerate(checks):
    cmds = common_cmds + [
        f"+seqs={seq}",
        f"+person={person}",
        f"+cameras={cam}",
    ]
    print(" ".join([f"{job}"] + cmds))
    subprocess.run(
        cmds, cwd="/private/home/taoshaf/Documents/projects/CondDenseDetection"
    )

0 sbatch launch/submit_jobs.bash runs/demo/run_on_egohumans.py +seqs=06_badminton/004_badminton +person=aria03 +cameras=cam11
Submitted batch job 51691525
1 sbatch launch/submit_jobs.bash runs/demo/run_on_egohumans.py +seqs=06_badminton/004_badminton +person=aria03 +cameras=cam12
Submitted batch job 51691526
2 sbatch launch/submit_jobs.bash runs/demo/run_on_egohumans.py +seqs=06_badminton/004_badminton +person=aria03 +cameras=cam13
Submitted batch job 51691527
3 sbatch launch/submit_jobs.bash runs/demo/run_on_egohumans.py +seqs=06_badminton/004_badminton +person=aria03 +cameras=cam14
Submitted batch job 51691528
4 sbatch launch/submit_jobs.bash runs/demo/run_on_egohumans.py +seqs=06_badminton/004_badminton +person=aria03 +cameras=cam15
Submitted batch job 51691529
5 sbatch launch/submit_jobs.bash runs/demo/run_on_egohumans.py +seqs=06_badminton/015_badminton +person=aria04 +cameras=cam02
Submitted batch job 51691530
6 sbatch launch/submit_jobs.bash runs/demo/run_on_egohumans.py +seqs=0